# Experimentos 18/10/23 - BayesFL

In [40]:
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pingouin as pg

import glob
import os
import shutil

pd.options.display.max_info_rows = 30000000
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 100)

sns.set_theme(style='darkgrid')

# Tamaño figuras
plt.rcParams["figure.figsize"] = (20,8)
plt.rcParams["figure.dpi"] = 600

pd.options.mode.chained_assignment = None

# Paleta de colores
color = ['#2a9d8f', '#e9c46a', '#f4a261', '#e76f51', '#b13c47']

color2 = ['#264653', '#2a9d8f', '#e9c46a', '#f4a261', '#e76f51', '#b13c47']

In [41]:
# Join all of csv files of the folder ../resultados, mantaining the header of the first file
path = r'../results/Client/' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)

df

,bbdd,algorithm,id,iteration,instances,threads,bdeu,SMHD,time(s)
0,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,1,5000,8,-54570.524663,10,0.154
1,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,2,5000,8,-53557.711947,7,0.039
2,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,3,5000,8,-53557.711947,7,0.031
3,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,4,5000,8,-53557.711947,7,0.037
4,alarm_50001,GES_BN_FusionIntersection_GES_BN_FusionInterse...,0,5,5000,8,-53557.711947,7,0.036
...,...,...,...,...,...,...,...,...,...
1212,barley_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,1,5000,8,-271816.301683,67,0.004
1213,barley_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,2,5000,8,-271769.371609,65,0.005
1214,barley_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,3,5000,8,-265302.774736,59,0.006
1215,barley_50004,pGES_BN_FusionUnion_None_BN_FusionUnion_fusion,3,4,5000,8,-265302.774736,59,0.004


In [42]:
# Replace the GES_BN_FusionIntersection_GES_build of column algorithm with four columns separated with the _
df[['algorithm','fusionClient','refinement','fusionServer','epoch']] = df.algorithm.str.split("_",expand=True,).drop(columns=[1,4])

df[['bbdd','sample']] = df.bbdd.str.split("_",expand=True,)

df

,bbdd,algorithm,id,iteration,instances,threads,bdeu,SMHD,time(s),fusionClient,refinement,fusionServer,epoch,sample
0,alarm,GES,0,1,5000,8,-54570.524663,10,0.154,FusionIntersection,GES,FusionIntersection,build,50001
1,alarm,GES,0,2,5000,8,-53557.711947,7,0.039,FusionIntersection,GES,FusionIntersection,build,50001
2,alarm,GES,0,3,5000,8,-53557.711947,7,0.031,FusionIntersection,GES,FusionIntersection,build,50001
3,alarm,GES,0,4,5000,8,-53557.711947,7,0.037,FusionIntersection,GES,FusionIntersection,build,50001
4,alarm,GES,0,5,5000,8,-53557.711947,7,0.036,FusionIntersection,GES,FusionIntersection,build,50001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1212,barley,pGES,3,1,5000,8,-271816.301683,67,0.004,FusionUnion,None,FusionUnion,fusion,50004
1213,barley,pGES,3,2,5000,8,-271769.371609,65,0.005,FusionUnion,None,FusionUnion,fusion,50004
1214,barley,pGES,3,3,5000,8,-265302.774736,59,0.006,FusionUnion,None,FusionUnion,fusion,50004
1215,barley,pGES,3,4,5000,8,-265302.774736,59,0.004,FusionUnion,None,FusionUnion,fusion,50004


### BDeu

In [43]:
# Drop the rows with epoch != build
df2 = df[df['epoch'] == 'build']

# Only iteration 5
df2 = df2[(df2['iteration'] == 1) | (df2['iteration'] == 5)]

df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "time(s)"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

bdeu  \
bbdd                                                                         alarm   
algorithm fusionClient       fusionServer       refinement iteration                 
GES       FusionIntersection FusionIntersection GES        1         -53993.785619   
                                                           5         -53653.577956   
                                                None       1         -53993.785619   
                                                           5         -53596.153387   
                             FusionUnion        GES        1         -53993.785619   
                                                           5         -53648.982757   
                                                None       1         -53993.785619   
                                                           5         -53605.576683   
          FusionUnion        FusionIntersection GES        1         -53993.785619   
                                                           5         -53671.570510   
                                                None       1         -53993.785619   
                                                           5         -53653.577956   
                             FusionUnion        GES        1         -53993.785619   
                                                           5         -53648.982757   
                                                None       1         -53993.785619   
                                                           5         -54030.288104   
pGES      FusionIntersection FusionIntersection GES        1         -53672.317608   
                                                           5         -53653.577956   
                                                None       1         -53651.755373   
                                                           5         -53596.153387   
                             FusionUnion        GES        1         -53672.317608   
                                                           5         -53672.317608   
                                                None       1         -53672.317608   
                                                           5         -53608.349299   
          FusionUnion        FusionIntersection GES        1         -53672.317608   
                                                           5         -53653.577956   
                                                None       1         -53672.317608   
                                                           5         -53653.577956   
                             FusionUnion        GES        1         -53672.317608   
                                                           5         -53672.317608   
                                                None       1         -53672.317608   
                                                           5         -53675.912854   

                                                                                     
bbdd                                                                         barley  
algorithm fusionClient       fusionServer       refinement iteration                 
GES       FusionIntersection FusionIntersection GES        1                    NaN  
                                                           5                    NaN  
                                                None       1                    NaN  
                                                           5                    NaN  
                             FusionUnion        GES        1                    NaN  
                                                           5                    NaN  
                                                None       1                    NaN  
                                                           5                    NaN  
          FusionUnion        FusionIntersection GES        1                    NaN  
                                   

### SMHD

In [44]:
# Drop the rows with epoch != build
df2 = df[df['epoch'] == 'build']

# Only iteration 5
df2 = df2[df2['iteration'] == 5]

df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "bdeu", "time(s)"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

SMHD  \
bbdd                                                                  alarm   
algorithm fusionClient       fusionServer       refinement iteration          
GES       FusionIntersection FusionIntersection GES        5           7.00   
                                                None       5           7.00   
                             FusionUnion        GES        5           7.25   
                                                None       5           7.00   
          FusionUnion        FusionIntersection GES        5           7.25   
                                                None       5           7.00   
                             FusionUnion        GES        5           7.25   
                                                None       5          11.00   
pGES      FusionIntersection FusionIntersection GES        5           7.00   
                                                None       5           7.00   
                             FusionUnion        GES        5           7.50   
                                                None       5           7.00   
          FusionUnion        FusionIntersection GES        5           7.00   
                                                None       5           7.00   
                             FusionUnion        GES        5           7.50   
                                                None       5           7.25   

                                                                             
bbdd                                                                 barley  
algorithm fusionClient       fusionServer       refinement iteration         
GES       FusionIntersection FusionIntersection GES        5            NaN  
                                                None       5            NaN  
                             FusionUnion        GES        5            NaN  
                                                None       5            NaN  
          FusionUnion        FusionIntersection GES        5            NaN  
                                                None       5            NaN  
                             FusionUnion        GES        5            NaN  
                                                None       5            NaN  
pGES      FusionIntersection FusionIntersection GES        5          57.25  
                                                None       5          55.25  
                             FusionUnion        GES        5          58.50  
                                                None       5          58.75  
          FusionUnion        FusionIntersection GES        5          57.25  
                                                None       5          57.25  
                             FusionUnion        GES        5          58.50  
                                                None       5          57.25

### Time(s)

In [45]:
# Drop the rows with epoch != build
df2 = df[df['epoch'] == 'build']

# Only iteration 5
df2 = df2[df2['iteration'] == 5]

df_unstack = df2.drop(["sample", "id", "threads", "instances", "epoch", "SMHD", "bdeu"], axis=1)
df_unstack.groupby(["bbdd", "algorithm", "fusionClient", "fusionServer", "refinement", "iteration"], observed=True).mean().unstack(0)

time(s)  \
bbdd                                                                    alarm   
algorithm fusionClient       fusionServer       refinement iteration            
GES       FusionIntersection FusionIntersection GES        5          0.03575   
                                                None       5          0.03925   
                             FusionUnion        GES        5          0.04050   
                                                None       5          0.03225   
          FusionUnion        FusionIntersection GES        5          0.04300   
                                                None       5          0.03450   
                             FusionUnion        GES        5          0.04350   
                                                None       5          0.04000   
pGES      FusionIntersection FusionIntersection GES        5          0.07300   
                                                None       5          0.06775   
                             FusionUnion        GES        5          0.07400   
                                                None       5          0.06750   
          FusionUnion        FusionIntersection GES        5          0.07400   
                                                None       5          0.08900   
                             FusionUnion        GES        5          0.07400   
                                                None       5          0.15800   

                                                                               
bbdd                                                                   barley  
algorithm fusionClient       fusionServer       refinement iteration           
GES       FusionIntersection FusionIntersection GES        5              NaN  
                                                None       5              NaN  
                             FusionUnion        GES        5              NaN  
                                                None       5              NaN  
          FusionUnion        FusionIntersection GES        5              NaN  
                                                None       5              NaN  
                             FusionUnion        GES        5              NaN  
                                                None       5              NaN  
pGES      FusionIntersection FusionIntersection GES        5          0.38500  
                                                None       5          0.42700  
                             FusionUnion        GES        5          0.47750  
                                                None       5          0.41975  
          FusionUnion        FusionIntersection GES        5          0.39125  
                                                None       5          0.38900  
                             FusionUnion        GES        5          0.41425  
                                                None       5          0.44950